### Exploratory

In [ ]:
import rp
import numpy as np
from icecream import ic 
from einops import rearrange, repeat
import torch

def get_video_mask(video):
    """ Given a THWC video, return a THW binary mask """
    assert rp.is_numpy_array(video)
    num_frames,height,width,num_channels=video.shape

    mask = np.zeros((num_frames,height,width),dtype=bool)
    
    #Binary matrix with shape (height, width)
    circle = rp.flat_circle_kernel(diameter=min(height,width))
    circle = rp.crop_image(circle,height,width,origin='center')
    circle = rp.as_binary_image(circle)
    mask[0]=circle
    # mask[5]=circle
    mask[len(mask)-1]=circle

    return mask

def get_all_mask_positions(mask):
    """
    Get (t, x, y) coordinates of all True pixels in the mask.
    """
    assert mask.ndim==3
    assert mask.dtype==bool
    
    ts, ys, xs= np.where(mask)
    #NOTE: np.where returns ys then xs, we return xs then ys!
    
    return np.stack([ts, xs, ys]).T # M×3

def get_random_mask_positions(mask, N):
    """
    Returns size [N, TXY]
    """
    assert mask.ndim==3
    assert mask.dtype==bool
    T,H,W = mask.shape
    
    all_positions = get_all_mask_positions(mask) # M×3
    chosen_positions = rp.random_batch_with_replacement(all_positions, N) #N×3
    chosen_positions = rp.as_numpy_array(chosen_positions)

    assert chosen_positions.shape == (N, 3)
    return chosen_positions #N×3

def get_random_mask_positions_per_frame(mask, N):
    T,H,W = mask.shape
    
    chosen_positions = get_random_mask_positions(mask, N)

    output = [[] for _ in range(T)]
    for t,x,y in chosen_positions:
        output[t].append([x,y])
    
    return output

def demo_get_random_mask_positions_per_frame():
    video = rp.load_video(
        "https://github.com/facebookresearch/co-tracker/raw/refs/heads/main/assets/apple.mp4",
        use_cache=True,
    )
    video = rp.resize_images(video, size=(480,720))
    video = rp.as_numpy_array(video)
    
    mask = get_video_mask(video)
    
    mask_positions_per_frame = get_random_mask_positions_per_frame(mask, 1000)
    
    dotted_video = []
    for mask_positions, frame in zip(mask_positions_per_frame, video):
        if len(mask_positions):
            x, y = rp.list_transpose(mask_positions) #Nx3 -> 3xN
            frame = rp.cv_draw_circles(frame, x, y, color='light blue', rim=2)
        dotted_video.append(frame)
    
    rp.display_video(rp.horizontally_concatenated_videos(mask,dotted_video), framerate=10)

demo_get_random_mask_positions_per_frame()

device = rp.select_torch_device(prefer_used=True)

In [ ]:
video = rp.load_video(
    "https://github.com/facebookresearch/co-tracker/raw/refs/heads/main/assets/apple.mp4",
    use_cache=True,
)
video = rp.resize_images(video, size=(480,720))
video = rp.as_numpy_array(video)


import dinkydata


mask = get_video_mask(video)

mask_positions_per_frame = get_random_mask_positions_per_frame(mask, 1000)

dotted_video = []
for mask_positions, frame in zip(mask_positions_per_frame, video):
    if len(mask_positions):
        x, y = rp.list_transpose(mask_positions) #Nx3 -> 3xN
        frame = rp.cv_draw_circles(frame, x, y, color='light blue', rim=2)
    dotted_video.append(frame)

rp.display_video(rp.horizontally_concatenated_videos(mask,dotted_video), framerate=10)

In [ ]:
# Sample 100 random points from the circle mask
track_points = get_random_mask_positions(mask, 1000) #100 x 3


# # Run cotracker with our custom points
tracks, visibility = rp.run_cotracker(
    video, 
    device=device,
    queries=track_points,  # Use our custom initialization points
    # grid_size = None,
)

In [ ]:
tracks.shape

In [ ]:
dotted_video = []
for frame, track in zip(video, tracks):
    x, y = track.T
    frame = rp.cv_draw_circles(frame, x, y, color='light blue', rim=2, )
    dotted_video.append(frame)
rp.display_video(dotted_video)

In [ ]:
# # Visualization functions
# def generate_uv_colors(tracks, frame_idx=0):
#     """Generate colors based on UV coordinates of initial positions."""
#     height, width = video[0].shape[:2]
    
#     colors = []
#     for x, y in tracks[frame_idx]:
#         x_py = to_python_num(x)
#         y_py = to_python_num(y)
#         # Generate colors based on UV coordinates
#         r = x_py/width  
#         g = y_py/height
#         b = 0
#         colors.append((r, g, b))
    
#     return colors

# def draw_tracks_on_video(video, tracks, colors, radius=5):
#     """Draw colored track points on video frames."""
#     result = []
#     for frame_number, frame in enumerate(video):
#         frame_copy = frame.copy()
#         for color, (x, y) in zip(colors, tracks[frame_number]):
#             x_py = to_python_num(x)
#             y_py = to_python_num(y)
            
#             frame_copy = cv_draw_circle(
#                 frame_copy,
#                 x_py, y_py,
#                 radius=radius,
#                 color=color,
#                 antialias=True,
#                 copy=False,
#             )
#         result.append(frame_copy)
#     return result

# # Apply visualization
# colors = generate_uv_colors(tracks)
# new_video = draw_tracks_on_video(video, tracks, colors, radius=8)
# display_video(new_video)
# print(f"SAVED: {save_video_mp4(new_video, 'circle_tracking.mp4')}")

In [ ]:
track_points.shape

In [ ]:
import dinkydata.counterfactual_dataset.counterfactual_dataset as dd
dd.demo()

In [ ]:
from rp import *
table = dd.index_table()
cartridge = rp.random_element(table).load()
rp.display_video(cartridge.preview())

In [ ]:

video = cartridge.orig_vid
gen_video = cartridge.gen_vid

mask = rp.resize_list(cartridge.mask, 49)
mask = rp.resize_images(mask, size=(480, 720))
mask = rp.as_grayscale_images(rp.as_binary_images(mask))

N=200
mask_positions_per_frame = get_random_mask_positions_per_frame(mask, N)
colors = [rp.as_rgb_float_color('random') for _ in range(N)]

# dotted_video = []
# for mask_positions, frame in zip(mask_positions_per_frame, video):
#     if len(mask_positions):
#         x, y = rp.list_transpose(mask_positions) #Nx3 -> 3xN
#         frame = rp.cv_draw_circles(frame, x, y, color='light blue', rim=2)
#     dotted_video.append(frame)

# rp.display_video(rp.horizontally_concatenated_videos(mask,dotted_video), framerate=10)

In [ ]:
# Sample 100 random points from the circle mask
track_points = get_random_mask_positions(mask, N) #100 x 3


# # Run cotracker with our custom points
tracks, visibility = rp.run_cotracker(
    video, 
    device="cuda",
    queries=track_points,  # Use our custom initialization points
    # grid_size = None,
)


# # Run cotracker with our custom points
tracks2, visibility2 = rp.run_cotracker(
    gen_video, 
    device="cuda",
    queries=track_points,  # Use our custom initialization points
    # grid_size = None,
)

In [ ]:
dotted_video = []
for frame, gen_frame, track, gen_track, vis, gen_vis, m in rp.eta(zip(
    video, gen_video, tracks, tracks2, visibility, visibility2, mask
),length=len(video)):
    x, y = rp.as_numpy_array(track.T)
    gx, gy = rp.as_numpy_array(gen_track.T)

    
    v = vis.T * gen_vis.T  # Only show when both are visibile
    radii = v * 5 + 1
    frame = rp.cv_draw_circles(frame, x, y, color=colors, rim=2, radius=radii)
    gen_frame = rp.cv_draw_circles(gen_frame, gx, gy, color=colors, rim=2, radius=radii)
    
    avg_frame = frame // 2 + gen_frame // 2
    avg_frame = rp.cv_draw_contours(avg_frame, [[[xi,yi],[xj,yj]] for xi,yi,xj,yj in zip(x,y,gx,gy)])
    dotted_frame = rp.horizontally_concatenated_images(
        rp.labeled_images(
            [
                frame,
                gen_frame,
                rp.blend_images(rp.blend_images(avg_frame, rp.auto_canny(m), mode="add"),m,.1,mode='add'),
            ],
            [
                "Original Video",
                "Generated Video",
                "Blended (for visualiztion)",
            ],
            font="R:Futura",
        )
    )
    dotted_video.append(dotted_frame)
rp.display_video(dotted_video,framerate=10)
rp.save_video_mp4(dotted_video, framerate=10)


### Final Func

In [ ]:
import rp
import torch
import numpy as np


def fast_scatter_add(output_tensor, latent_tracks, track_colors, num_timesteps, num_points, width, height):
    """
    Efficiently adds tracking point colors to a latent tensor using scatter_add.
    
    Analogy: 
    The original implementation is like manually placing colored dots on a canvas one at a time:
    ```
    for lt in range(LT):
        for n in range(N):
            color = track_colors[n]
            x, y = latent_tracks[lt, n].long()
            if 0<=x<LW and 0<=y<LH:
                dotted_latent[lt, :, y, x] += color
    ```
    
    This optimized version is like having a machine that can place all dots of the same color
    simultaneously at their respective coordinates.
    
    Args:
        output_tensor: Zero-initialized tensor to populate with values (LT, LC, LH, LW)
        latent_tracks: Tensor containing track coordinates (LT, N, 2)
        track_colors: Tensor of colors for each track point (N, LC)
        num_timesteps: Number of timesteps (LT)
        num_points: Number of track points (N)
        width: Width of latent tensor (LW)
        height: Height of latent tensor (LH)
        
    Returns:
        torch.Tensor: Populated tensor with added track colors
    """
    device = output_tensor.device
    dtype = output_tensor.dtype
    C = track_colors.shape[1]  # Number of channels
    
    # Extract x and y coordinates
    xs, ys = latent_tracks[..., 0], latent_tracks[..., 1]
    
    # Create mask for valid coordinates (within bounds)
    valid_mask = (xs >= 0) & (xs < width) & (ys >= 0) & (ys < height)
    
    # Loop over timesteps (still needed, but with vectorized inner operations)
    for t in range(num_timesteps):
        # Get valid points for this timestep
        t_valid = valid_mask[t]
        if not t_valid.any():
            continue
            
        # Get coordinates of valid points
        valid_xs = xs[t, t_valid]
        valid_ys = ys[t, t_valid]
        valid_indices = t_valid.nonzero().squeeze(1)
        
        # Get colors for valid points
        valid_colors = track_colors[valid_indices]
        
        # Compute indices for scatter operation
        indices = valid_ys * width + valid_xs
        
        # Perform scatter_add for each channel
        for c in range(C):
            # Create flat output tensor for this timestep and channel
            flat_output = output_tensor[t, c].view(-1)
            
            # Scatter add the values
            flat_output.scatter_add_(0, indices, valid_colors[:, c])
    
    return output_tensor

def get_all_mask_positions(mask):
    """
    Get (t, x, y) coordinates of all True pixels in the mask.
    
    Args:
        mask: A 3D boolean mask of shape (T, H, W) where True indicates positions to extract.
              Can be either torch.Tensor or numpy.ndarray.
    
    Returns:
        torch.Tensor: Tensor of shape (M, 3) containing (t, x, y) coordinates of all True pixels,
                      where M is the number of True pixels in the mask
    """
    assert mask.ndim == 3
    assert mask.dtype == bool or mask.dtype == torch.bool
    
    if isinstance(mask, torch.Tensor):
        # Already a torch tensor, use torch.where
        indices = torch.where(mask)
        ts, ys, xs = indices
        # Stack to get M×3 tensor
        return torch.stack([ts, xs, ys], dim=1)
    else:
        # Convert numpy array to torch tensor
        mask_tensor = torch.from_numpy(mask)
        indices = torch.where(mask_tensor)
        ts, ys, xs = indices
        #NOTE: torch.where (like np.where) returns ys then xs, we return xs then ys!
        return torch.stack([ts, xs, ys], dim=1)

def get_random_mask_positions(mask, N):
    """
    Randomly samples N positions from the True pixels in a 3D mask.
    
    Args:
        mask: A 3D boolean mask of shape (T, H, W) where True indicates valid positions.
              Can be either torch.Tensor or numpy.ndarray.
        N: Number of random positions to sample with replacement
    
    Returns:
        torch.Tensor: Tensor of shape (N, 3) containing randomly sampled (t, x, y) coordinates
    """
    assert mask.ndim == 3
    assert mask.dtype == bool or mask.dtype == torch.bool
    T, H, W = mask.shape
    
    # Get all positions as a torch tensor
    all_positions = get_all_mask_positions(mask)  # M×3 torch tensor
    
    if len(all_positions) == 0:
        raise ValueError("Mask contains no True values to sample from")
    
    # Determine device based on the tensor
    device = all_positions.device
    
    # Randomly sample N indices with replacement
    idx = torch.randint(0, len(all_positions), (N,), device=device)
    chosen_positions = all_positions[idx]
    
    assert chosen_positions.shape == (N, 3)
    return chosen_positions  # N×3 torch tensor

def generate_dotted_latents(
    videos,
    latent_mask,
    *,
    out_channels = 16, 
    num_points=1024,
    device = None,
    silent = True,
):
    """
    Generates a latent-tracking-point control video from a source video
    Uses CoTracker to move random dots around a latent video

    Args:
        - videos: Give a list of videos, either in torch BTCHW form or a list of rp videos
        - latent_mask: Determines the duration and dimensions of the output, 
                       and where tracking points are initialized.
                       Either numpy arrays or torch tensors are ok.
                       In THW form. 
        - out_channels: Determines the number of channels in the output
        - num_points: The number of tracking points we use
        - device: Optional, if specified we use that torch device.
        - silent: If False, will print debug info.

    Returns:
        - torch.Tensor: BTCHW form (where B comes from videos, THW come from latent_mask, and C comes from out_channels)
    """
    dtype = torch.bfloat16
    device = device or rp.select_torch_device(prefer_used=True, silent=silent, reserve=True)

    LT, LH, LW = latent_mask.shape
    LC = out_channels
    N = num_points
        
    videos = rp.as_torch_videos(videos, device=device, dtype=dtype)
    B, VT, _, VH, VW = videos.shape

    spatial_scale = rp.assert_equality(VH/LH, VW/LW)

    #latent_mask -> mask has same dimensions and duration as video, and is bool
    mask = latent_mask
    mask = rp.resize_list(mask, VT)
    mask = rp.as_numpy_array(mask)
    mask = rp.as_grayscale_images(mask)
    mask = rp.as_binary_images(mask)
    mask = rp.resize_images(mask, size=(VH, VW), interp='nearest')
    mask = rp.as_binary_images(mask)
    assert rp.is_numpy_array(mask)

    # Sample N random spatio-temporal points from the mask
    track_points = get_random_mask_positions(mask, N)
    # Convert to numpy array since run_cotracker expects numpy
    track_points = track_points.cpu().numpy() if isinstance(track_points, torch.Tensor) else track_points
    track_colors = torch.randn(N, LC, device=device, dtype=dtype)

    dotted_latents = []
    for video in videos:
        #TODO: Can we get away with simply tracking less frames instead of tracking all frames and squashing time afterwards?
        tracks, visibility = rp.run_cotracker(
            video, 
            device=device,
            queries=track_points,
        )

        latent_tracks = rp.resize_list(tracks, LT) // spatial_scale #Scale it down spatiotemporally
        latent_tracks = latent_tracks.long()
        
        dotted_latent = torch.zeros(LT, LC, LH, LW, device=device, dtype=dtype)
        
        # Use optimized scatter_add helper function
        dotted_latent = fast_scatter_add(dotted_latent, latent_tracks, track_colors, LT, N, LW, LH)
        
        dotted_latents.append(dotted_latent)
    dotted_latents = torch.stack(dotted_latents)
    
    rp.validate_tensor_shapes(
        videos        ='torch: B VT 3  VH VW',
        dotted_latents='torch: B LT LC LH LW',
        dotted_latent ='torch:   LT LC LH LW',
        video         ='torch:   VT 3  VH VW',
        mask          ='numpy:   VT    VH VW',
        tracks        ='torch:  VT N XY',
        visibility    ='torch:  VT N',
        track_points  ='numpy:     N TXY', 
        latent_tracks ='torch: LT  N XY',
        latent_mask   ='       LT    LH LW',
        track_colors  ='torch: N  LC',
        XY  = 2,
        TXY = 3,
        **rp.gather_vars('N LC LT LH LW B VT VH VW'), #Already decided
        verbose = not silent and 'bold white random green',
    )

    #CogVideoX Assumptions
    assert spatial_scale==8          , "Assuming we're using CogVideoX"
    assert (LT, LH, LW)==(13, 60, 90), "Assuming we're using CogVideoX"
    assert (VT, VH, VW)==(49,480,720), "Assuming we're using CogVideoX"

    return rp.gather_vars('dotted_latents')

def demo_dotted_latents(*video_urls):
    video_urls = rp.detuple(video_urls) or ["https://video-previews.elements.envatousercontent.com/23ce1f71-c55d-4bc3-bfad-bc7bf8d8168a/watermarked_preview/watermarked_preview.mp4"]
    videos = rp.load_video(video_urls, use_cache=True)
    videos = rp.resize_videos(videos,size=(480,720))
    videos = rp.resize_lists(videos, length=49)

    rp.tic()
    result = generate_dotted_latents(
        [video],
        np.ones((13, 60, 90)),
        out_channels=3,
        silent=False,
    )
    [dotted_latent_video] = result.dotted_latents
    rp.ptoc("generate_dotted_latents")
    
    rp.display_video(
        rp.horizontally_concatenated_videos(
            rp.resize_list(video, 13),
            rp.as_numpy_images(
                rp.torch_resize_images(
                    dotted_latent_video, size=8, interp="nearest", copy=True
                )
                / 5
                + 0.5
            ),
        ),
        framerate=10,
    )


In [ ]:
demo_dotted_latents()